In [100]:
from rich.console import Console
from dotenv import load_dotenv
from openai import OpenAI
import os
import requests
import json
load_dotenv(override=True)


True

In [101]:
def show(text):
    try:
        Console().print(text)
    except Exception:
        print(text)

In [102]:
DEEPSEEK_URL="https://api.deepseek.com/v1"
openai=OpenAI(base_url=DEEPSEEK_URL,api_key=os.getenv("DEEPSEEK_API_KEY"))

In [103]:
todos=[]
completed=[]

In [104]:
def get_todo_report() -> str:
    result=""
    for index,todo in enumerate(todos):
        if completed[index]:
            result+=f"Todo #{index+1}: [green][strike]{todo}[/strike][/green]\n"
        else:
            result +=f"Todo #{index+1}:{todo}\n"
    show(result)
    return result

In [105]:
get_todo_report()

''

In [106]:
def create_todos(descriptions: list[str])-> str:
    todos.extend(descriptions)
    completed.extend([False]* len(descriptions))
    return get_todo_report()

In [107]:
def mark_complete(index: int,completion_notes:str)->str:
    if 1<=index <=len(todos):
        completed[index-1]=True
    else:
        return "No todo at this index."
    Console().print(completion_notes)
    return get_todo_report()

In [108]:
todos,completed=[],[]

create_todos(["Buy groceries","Finish extra lab","Eat banana"])

Todo #1:Buy groceries
Todo #2:Finish extra lab
Todo #3:Eat banana

'Todo #1:Buy groceries\nTodo #2:Finish extra lab\nTodo #3:Eat banana\n'

In [109]:
mark_complete(1,"bought")

bought

Todo #1: Buy groceries
Todo #2:Finish extra lab
Todo #3:Eat banana

'Todo #1: [green][strike]Buy groceries[/strike][/green]\nTodo #2:Finish extra lab\nTodo #3:Eat banana\n'

In [110]:
create_todos_json = {
    "name": "create_todos",
    "description": "Add new todos from a list of descriptions and return the full list",
    "parameters": {
        "type": "object",
        "properties": {
            "descriptions": {
                'type': 'array',
                'items': {'type': 'string'},
                'title': 'Descriptions'
                }
            },
        "required": ["descriptions"],
        "additionalProperties": False
    }
}

In [111]:
mark_complete_json = {
    "name": "mark_complete",
    "description": "Mark complete the todo at the given position (starting from 1) and return the full list",
    "parameters": {
        'properties': {
            'index': {
                'description': 'The 1-based index of the todo to mark as complete',
                'title': 'Index',
                'type': 'integer'
                },
            'completion_notes': {
                'description': 'Notes about how you completed the todo in rich console markup',
                'title': 'Completion Notes',
                'type': 'string'
                }
            },
        'required': ['index', 'completion_notes'],
        'type': 'object',
        'additionalProperties': False
    }
}

In [112]:
tools=[{'type':'function','function':create_todos_json},
{'type':'function','function':mark_complete_json}]

In [113]:
from ast import arguments


def handle_tool_calls(tool_calls):
    results=[]
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments=json.loads(tool_call.function.arguments)
        tool=globals().get(tool_name)
        result=tool(**arguments) if tool else {}
        results.append({"role":"tool","content":json.dumps(result),"tool_call_id":tool_call.id})
    return results

In [114]:
def loop(messages):
    done = False
    while not done:
        response = openai.chat.completions.create(model="deepseek-chat", messages=messages, tools=tools, reasoning_effort="none")
        finish_reason = response.choices[0].finish_reason
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    show(response.choices[0].message.content)

In [115]:
system_message = """
You are given a problem to solve, by using your todo tools to plan a list of steps, then carrying out each step in turn.
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution.
If any quantity isn't provided in the question, then include a step to come up with a reasonable estimate.
Provide your solution in Rich console markup without code blocks.
Do not ask the user questions or clarification; respond only with the answer after using your tools.
"""
user_message = """"
A train leaves Boston at 2:00 pm traveling 60 mph.
Another train leaves New York at 3:00 pm traveling 80 mph toward Boston.
When do they meet?
"""
messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]

In [116]:
todos,completed=[],[]
loop(messages)

Todo #1:Define the problem: Two trains traveling toward each other from Boston and New York
Todo #2:Estimate the distance between Boston and New York
Todo #3:Set up equations for train positions over time
Todo #4:Solve for meeting time
Todo #5:Convert solution to readable time format

**Problem Definition:**
- Train A leaves Boston at 2:00 pm traveling 60 mph toward New York
- Train B leaves New York at 3:00 pm traveling 80 mph toward Boston
- They are traveling toward each other
- Need to find when they meet

Todo #1: Define the problem: Two trains traveling toward each other from Boston and New York
Todo #2:Estimate the distance between Boston and New York
Todo #3:Set up equations for train positions over time
Todo #4:Solve for meeting time
Todo #5:Convert solution to readable time format

**Distance Estimate:**
The actual driving distance between Boston and New York is approximately 215 miles. However, for train travel, the 
distance is typically around 190-200 miles. I'll use **200 miles** as a reasonable estimate for the rail distance 
between Boston and New York.

Todo #1: Define the problem: Two trains traveling toward each other from Boston and New York
Todo #2: Estimate the distance between Boston and New York
Todo #3:Set up equations for train positions over time
Todo #4:Solve for meeting time
Todo #5:Convert solution to readable time format

...

**Solving the equation:**

Equation: 60t + 80(t - 1) = 200

Expand: 60t + 80t - 80 = 200
Combine: 140t - 80 = 200
Add 80 to both sides: 140t = 280
Divide by 140: t = 2

**Solution:** t = 2 hours after 2:00 pm

**Check:**
- Train from Boston travels: 60 mph × 2 hours = 120 miles
- Train from New York travels: 80 mph × (2 - 1) hours = 80 mph × 1 hour = 80 miles
- Total distance covered: 120 + 80 = 200 miles ✓

Todo #1: Define the problem: Two trains traveling toward each other from Boston and New York
Todo #2: Estimate the distance between Boston and New York
Todo #3: Set up equations for train positions over time
Todo #4: Solve for meeting time
Todo #5:Convert solution to readable time format

**Time Conversion:**

Starting time: 2:00 pm
Meeting time: 2:00 pm + 2 hours = **4:00 pm**

**Final Answer:** The trains meet at **4:00 pm**.

Todo #1: Define the problem: Two trains traveling toward each other from Boston and New York
Todo #2: Estimate the distance between Boston and New York
Todo #3: Set up equations for train positions over time
Todo #4: Solve for meeting time
Todo #5: Convert solution to readable time format

## Solution

**The trains meet at 4:00 pm.**

**Step-by-step reasoning:**

1. **Distance:** Estimated the rail distance between Boston and New York as 200 miles.

2. **Timing:** 
   - Boston train leaves at 2:00 pm traveling 60 mph
   - New York train leaves at 3:00 pm traveling 80 mph

3. **Equation setup:** Let t = hours after 2:00 pm
   - Boston train distance: 60t miles
   - New York train distance: 80(t - 1) miles (since it leaves 1 hour later)
   - Meeting condition: 60t + 80(t - 1) = 200

4. **Solving:** 
   60t + 80t - 80 = 200
   140t = 280
   t = 2 hours after 2:00 pm

5. **Verification:**
   - Boston train travels: 60 × 2 = 120 miles
   - New York train travels: 80 × 1 = 80 miles
   - Total: 120 + 80 = 200 miles ✓

**Therefore, the trains meet at 4:00 pm.**